<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/Test_Jo_brachn/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [26]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

In [27]:
import pandas as pd
import math
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [28]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

In [29]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [30]:
chargers.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [31]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [32]:
dtrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [33]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [34]:
etrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [35]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [36]:
routes

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00
5,s-3,Ditzingen,120,32,09:00,16:00
6,s-4,Ditzingen,120,32,06:30,16:30
7,w1,Ditzingen,100,32,05:30,15:30
8,w2,Ditzingen,100,32,08:00,18:00
9,w3,Ditzingen,100,32,06:45,16:45


### Indexmengen

In [37]:
#Erstellen einer Modellinstanz
scip = Model()

In [38]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(0, 96) # 96 Zeitintervalle pro Tag
I = range(1, 4) # Potenzielle Säulen-Slots
# T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
# print("T_night (Nachtintervalle):", list(T_night))


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
P (potenzielle Fahrzeuge): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
T (Zeitintervalle): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]
I (Säulen): [1, 2, 3]


### Parameter

In [39]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    cons_e[m]    = row["avg_energy_kWh_per_100km"]
    thg_e[m]     = row["thg_yearly"]
    pmax_veh[m]  = row["max_power"]
    batt_kwh[m]  = row["soc_max_kWh"]

batt_kwh['ActrosL'] = 0
pmax_veh['ActrosL'] = 0

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[m] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[m] = row["avg_diesel_per_100km"]

cons_e['ActrosL'] = 0
print(cons_e)

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden
diesel = 1.60
SOC_T_Start = 0

# Netz & Tarif
P_grid_max = 500.0 # max kW am Depot
P_grid_add = 500.0 # Zusatzleistung bei Netzausbau
capex_grid_add = 10000.0 # Kosten/Jahr für Netzausbau

c_energy = 0.25   # Arbeitspreis Strom in €/kWh
c_capex = 1000.0   # Stromkosten in €/a
c_peak = 150.0    # Leistungspreis in €/kW

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_capex_bat_kW = 30.0      # Batteriekosten in €/kW
c_capex_bat_kWh = 350.0     # Batteriekosten in €/kWh
roundtrip_eff = 0.98 # Round-Trip Efficiency aka Wirkungsgrad
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975 # Max. Entladetiefe
soc_bat_min_frac = 1.0 - dod    # = 0.025


# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print(E_route_e)

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

{'eActros600': 110, 'eActros400': 105, 'ActrosL': 0}
Beispiel dist_total[r]: [('t-4', 250), ('t-5', 250), ('t-6', 250)]

Charger pmax (kW): {'Alpitronic-50': 50, 'Alpitronic-200': 200, 'Alpitronic-400': 400}

e-Lkw batt_kwh: {'eActros600': 621, 'eActros400': 414, 'ActrosL': 0}

Energiebedarf Beispiel (erste Route, e400/e600):
  ('t-4', 'eActros600') = 275.0 kWh
  ('t-4', 'eActros400') = 262.5 kWh
  ('t-4', 'ActrosL') = 0.0 kWh
{('t-4', 'eActros600'): 275.0, ('t-4', 'eActros400'): 262.5, ('t-4', 'ActrosL'): 0.0, ('t-5', 'eActros600'): 275.0, ('t-5', 'eActros400'): 262.5, ('t-5', 'ActrosL'): 0.0, ('t-6', 'eActros600'): 275.0, ('t-6', 'eActros400'): 262.5, ('t-6', 'ActrosL'): 0.0, ('s-1', 'eActros600'): 132.0, ('s-1', 'eActros400'): 126.0, ('s-1', 'ActrosL'): 0.0, ('s-2', 'eActros600'): 132.0, ('s-2', 'eActros400'): 126.0, ('s-2', 'ActrosL'): 0.0, ('s-3', 'eActros600'): 132.0, ('s-3', 'eActros400'): 126.0, ('s-3', 'ActrosL'): 0.0, ('s-4', 'eActros600'): 132.0, ('s-4', 'eActros400'): 126.0

## Hilfsfunktionen

In [40]:
def time_to_t_interval(uhrzeit):
  # Den String am Doppelpunkt trennen
  stunden_str, minuten_str = uhrzeit.split(":")
  # In Zahlen (Integer) umwandeln
  stunden = int(stunden_str)
  minuten = int(minuten_str)
  t = stunden * 4
  t = t + minuten / 15
  return t

## Weitere Parameter

In [41]:
# Verbrauch pro Zeitintervall pro LKW Art
# Erweitere deine Verbrauchsmatrix
verbrauch_pro_intervall = {}
for r in R:
    # 1. Daten holen (aus deinen Routen-Daten)
    dist = dist_total[r]
    start = time_to_t_interval(start_time[r])
    ende = time_to_t_interval(end_time[r])
    dauer = ende - start

    for l in L:
        # 2. Verbrauch pro Intervall berechnen
        if l in ['eActros400', 'eActros600']:
            # Elektro: Distanz * Verbrauch / Dauer
            #print(dist, cons_e[l], dauer)
            kwh_pro_t = (dist/100 * cons_e[l]) / dauer
        else:
            # Diesel: Verbraucht 0 kWh Strom
            kwh_pro_t = 0

        # 3. In Matrix speichern
        for t in T:
            if start <= t < ende:
                verbrauch_pro_intervall[r, l, t] = kwh_pro_t
            else:
                verbrauch_pro_intervall[r, l, t] = 0

{('t-4', 'ActrosL', 0): 0, ('t-4', 'ActrosL', 1): 0, ('t-4', 'ActrosL', 2): 0, ('t-4', 'ActrosL', 3): 0, ('t-4', 'ActrosL', 4): 0, ('t-4', 'ActrosL', 5): 0, ('t-4', 'ActrosL', 6): 0, ('t-4', 'ActrosL', 7): 0, ('t-4', 'ActrosL', 8): 0, ('t-4', 'ActrosL', 9): 0, ('t-4', 'ActrosL', 10): 0, ('t-4', 'ActrosL', 11): 0, ('t-4', 'ActrosL', 12): 0, ('t-4', 'ActrosL', 13): 0, ('t-4', 'ActrosL', 14): 0, ('t-4', 'ActrosL', 15): 0, ('t-4', 'ActrosL', 16): 0, ('t-4', 'ActrosL', 17): 0, ('t-4', 'ActrosL', 18): 0, ('t-4', 'ActrosL', 19): 0, ('t-4', 'ActrosL', 20): 0, ('t-4', 'ActrosL', 21): 0, ('t-4', 'ActrosL', 22): 0, ('t-4', 'ActrosL', 23): 0, ('t-4', 'ActrosL', 24): 0, ('t-4', 'ActrosL', 25): 0, ('t-4', 'ActrosL', 26): 0, ('t-4', 'ActrosL', 27): 0, ('t-4', 'ActrosL', 28): 0, ('t-4', 'ActrosL', 29): 0, ('t-4', 'ActrosL', 30): 0, ('t-4', 'ActrosL', 31): 0, ('t-4', 'ActrosL', 32): 0, ('t-4', 'ActrosL', 33): 0, ('t-4', 'ActrosL', 34): 0, ('t-4', 'ActrosL', 35): 0, ('t-4', 'ActrosL', 36): 0, ('t-4', 'A

In [42]:
route_aktiv = {}
for r in R:
    start_t = time_to_t_interval(start_time[r])
    ende_t = time_to_t_interval(end_time[r])
    for t in T:
        if start_t <= t < ende_t:
            route_aktiv[r, t] = 1
        else:
            route_aktiv[r, t] = 0

## Entscheidungsvariablen

In [44]:
Flottenwahl = {}
for l in L:
    for p in P:
        Flottenwahl[l, p] = scip.addVar(vtype = 'B', name =f"LKW_{l}_auf_ID_{p}")

Zuordnung_LKW_Route = {}
for r in R:
    for p in P:
        Zuordnung_LKW_Route[r, p] = scip.addVar(vtype = 'B', name =f"Route_{r}_wird_von_LKW_mit_ID_{p}_gefahren")

Auswahl_Charger = {}
for c in C:
    for i in I:
        Auswahl_Charger[c, i] = scip.addVar(vtype = 'B', name =f"Charger_{c}_ist_Säule_Nummer_{i}")

Zuordnung_LKW_Zeitpunkt_Charger = {}
for p in P:
    for t in T:
        for i in I:
            Zuordnung_LKW_Zeitpunkt_Charger[p,t,i] = scip.addVar(vtype = 'B', name =f"LKW_{l}_zum_Zeitpunkt_{t}_an_Charger_{c}")

SOC_Zeit_Fahrzeug = {}
Ladeleistung = {} # Von p verwendete Ladeleistung zum Zeitpunkt t
for p in P:
  for t in T:
    SOC_Zeit_Fahrzeug[p,t] = scip.addVar(vtype = 'C',lb=0, name=f"SOC_FahrzeugID_{p}_Zeitpunkt{t}")
    Ladeleistung[p, t] = scip.addVar(vtype='C', lb=0, name=f"Ladeleistung_{p}_{t}")

# Batteriespeicher
SOC_Speicher = {}      # Aktueller Stand in kWh
Laden_Speicher = {}    # Leistung vom Netz in den Speicher (kW)
Entladen_Speicher = {} # Leistung vom Speicher ins Depot/Lkw (kW)

for t in T:
    SOC_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"SOC_Speicher_t{t}")
    Laden_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"Laden_Speicher_t{t}")
    Entladen_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"Entladen_Speicher_t{t}")


Einsatz = {}
for r in R:
    for l in L:
        for p in P:
            # Diese Variable ist 1, wenn ID p Route r fährt UND Typ l ist
            Einsatz[r, l, p] = scip.addVar(vtype='B', name=f"Einsatz_{r}_{l}_{p}")


Ausbau_Netz = scip.addVar(vtype='B', name="Netzausbau")

Ausbau_Bat = scip.addVar(vtype='B', name="Batterie_kaufen")

Batterie_kWh = scip.addVar(vtype='C', lb=0, name="Batteriekapa_kWh")
Batterie_kW = scip.addVar(vtype='C', lb=0, name="Batterieleistung_kW")

P_peak = scip.addVar(vtype='C', lb=0, name="Jahres_Leistungsspitze_kW")

len(scip.getVars())



11562

## Restriktionen

In [45]:
#Für Einsatz{}
for r in R:
    for l in L:
        for p in P:
            # Wenn Einsatz=1, MUSS Zuordnung=1 sein
            scip.addCons(Einsatz[r, l, p] <= Zuordnung_LKW_Route[r, p])
            # Wenn Einsatz=1, MUSS Flottenwahl=1 sein
            scip.addCons(Einsatz[r, l, p] <= Flottenwahl[l, p])
            # Wenn beide 1 sind, MUSS Einsatz 1 werden
            scip.addCons(Einsatz[r, l, p] >= Zuordnung_LKW_Route[r, p] + Flottenwahl[l, p] - 1)
print("1.")
print (scip.getNConss())

# Zuordnung_LKW_Route{}
#1. Jede Route muss genau einmal bedient werden
for r in R:
  scip.addCons(quicksum(Zuordnung_LKW_Route[r,p] for p in P)==1)


#2. Jede Route r braucht ein Fahrzeug p, und dieses Fahrzeug p MUSS ein Modell l haben
for r in R:
    for p in P:
        # Wenn Route r dem Fahrzeug p zugeordnet ist (Variable == 1),
        # dann muss die Summe der Flottenwahl über alle Modelle l für dieses p ebenfalls 1 sein.
        scip.addCons(
            Zuordnung_LKW_Route[r, p] <= quicksum(Flottenwahl[l, p] for l in L),
            name=f"Route_braucht_Modell_{r}_{p}"
        )

print("2.")
print (scip.getNConss())
# Flottenwahl{}
#1. Max 20 Fahrzeuge
scip.addCons(quicksum(Flottenwahl[l,p] for p in P for l in L)<=20)

#2. Jede ID hat 1 oder weniger(0) LKW Modelle
for p in P:
  scip.addCons(quicksum(Flottenwahl[l,p] for l in L)<=1)

print("3.")
print (scip.getNConss())
# SOC_Zeit_Fahrzeug{}
#1. Berechnung SOC zum Zeitpunkt t
for p in P:
    for t in T:
        if t == 0:
            # Setzt den SOC zum Zeitpunkt 0 fest auf SOC_T_Start
            scip.addCons(SOC_Zeit_Fahrzeug[p, t] == SOC_T_Start, name=f"Start_SOC_{p}")
        else:
            # Für alle anderen Zeitpunkte gilt die normale Bilanz
            t_prev = t - 1
            energie_geladen = Ladeleistung[p, t] * 0.25
            energie_verbrauch = quicksum(Einsatz[r, l, p] * verbrauch_pro_intervall[r, l, t] for r in R for l in L)

            scip.addCons(SOC_Zeit_Fahrzeug[p, t] == SOC_Zeit_Fahrzeug[p, t_prev] + energie_geladen - energie_verbrauch,
                         name=f"SOC_Update_{p}_{t}")

#2. Regel dass SOC am Anfang des Tages[t=0] = SOC Ende des Tages[t=95] sein muss
for p in P:
  scip.addCons(SOC_Zeit_Fahrzeug[p,0] == SOC_Zeit_Fahrzeug[p, 95])

#3. SOC <= Max.Kappa LKW
for p in P:
    for t in T:
        # Der SOC von LKW p zum Zeitpunkt t darf die Kapazität
        # des gewählten Modells l nicht überschreiten.
        scip.addCons(
            SOC_Zeit_Fahrzeug[p, t] <= quicksum(Flottenwahl[l, p] * batt_kwh[l] for l in L),
            name=f"Max_Kapazitaet_LKW_{p}_t{t}"
        )

# 4. SOC niemals < 0 bereits über lower bound definiert


# Route_Aktiv{}
# 1. Keine_Ueberlappung
for p in P:
    for t in T:
        # Die Summe aller Routen r, die zum Zeitpunkt t aktiv sind,
        # darf für LKW p nicht größer als 1 sein.
        scip.addCons(
            quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R) <= 1,
            name=f"Keine_Ueberlappung_LKW_{p}_t{t}"
        )
print("4.")
print (scip.getNConss())
# Ladeleistung [p,t]
# 1. Nur Laden wenn am Depot / nicht auf Tour
for p in P:
    for t in T:
        # Summe ist 1, wenn der LKW fährt, 0 wenn er im Depot ist
        ist_auf_tour = quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

        # Wenn ist_auf_tour == 1, muss Ladeleistung <= 0 sein (also 0)
        # Wenn ist_auf_tour == 0, darf er laden (begrenzt durch einen großen Wert/M-Faktor)
        scip.addCons(Ladeleistung[p, t] <= (1 - ist_auf_tour) * 500,
                     name=f"Laden_nur_im_Depot_{p}_{t}")

#2. Wenn Ladeleistung > 0 Dann muss an Säule stehen
for p in P:
    for t in T:
        # Wenn Ladeleistung > 0, dann MUSS die Zuordnung zu IRGENDEINER Säule 1 sein
        scip.addCons(
            Ladeleistung[p, t] <= 500 * quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for i in I),
            name=f"Strom_nur_mit_Stecker_{p}_{t}"
        )

#3. Max_Power von LKW-Model
for p in P:
    for t in T:
        # Die Ladeleistung darf die fahrzeugspezifische Grenze nicht überschreiten
        # max_ladeleistung_lkw = {'eActros400': 150, 'eActros600': 375, 'Diesel_Actros': 0}
        scip.addCons(Ladeleistung[p, t] <= quicksum(Flottenwahl[l, p] * pmax_veh[l] for l in L),
                     name=f"Max_Ladeleistung_Fahrzeug_{p}_{t}")

# 4. Max leistung je Charger
for i in I: # Für jede physische Säule
    for t in T: # Zu jedem Zeitpunkt
        # Die Summe der Ladeleistung aller LKWs an dieser einen Säule i
        # muss kleiner gleich der Kapazität der gewählten Säule sein.
        scip.addCons(
            quicksum(Ladeleistung[p, t] for p in P if (p, t, i) in Zuordnung_LKW_Zeitpunkt_Charger)
            <= quicksum(Auswahl_Charger[c, i] * pmax_ch[c] for c in C),
            name=f"Saeulenkapazitaet_i{i}_t{t}"
        )


# 5. Angepasster Netzanschluss (inkl. Speicher)
for t in T:
    # Strom vom Netz wird verbraucht durch:
    # + Laden der LKWs
    # + Laden des Speichers
    # - Entladen des Speichers (liefert Strom ins Depot, entlastet das Netz)
    scip.addCons(
        quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]
        <= 500 + (500 * Ausbau_Netz),
        name=f"Netzanschluss_Limit_mit_Speicher_t{t}"
    )
print("5.")
print (scip.getNConss())
#Zuordnung_LKW_Zeitpunkt_Charger[p,t,i]
#1. Max 2 LKW pro Säule und Säule muss existieren
for i in I:
    for t in T:
        scip.addCons(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for p in P)
            <= 2 * quicksum(Auswahl_Charger[c, i] for c in C),
            name=f"Ports_nur_bei_existierender_Saeule_{i}_t{t}"
        )
#2. Ein LKW an max 1 Säule laden
for p in P:
    for t in T:
        # Ein LKW p kann zum Zeitpunkt t an maximal einer Säule i hängen
        scip.addCons(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for i in I) <= 1,
            name=f"Max_Eine_Saeule_Pro_LKW_{p}_{t}"
        )

# 3.Säule nicht wechseln
for p in P:
    for t in T:
        if t < 95: # Nicht für das letzte Intervall (t+1 wäre sonst out of range)
            for i in I:
                # Logik: Wenn (LKW an Säule i zu t) UND (LKW lädt zu t+1),
                # dann MUSS (LKW an Säule i zu t+1)
                # Mathematisch über eine lineare Bedingung:
                scip.addCons(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t+1, j] for j in I if j != i)
                    <= 1,
                    name=f"Kein_Saeulenwechsel_{p}_{t}_{i}"
                )

# 4.
# Definition der Nachtzeit (18:00 bis 06:00 Uhr)
# 18:00 Uhr ist Intervall 72, 06:00 Uhr ist Intervall 24
T_Nacht = [t for t in T if t >= 72 or t < 24]

for p in P:
    for t in T_Nacht:
        t_next = (t + 1) % 96
        # Nur wenn das nächste Intervall auch noch in der Nacht liegt
        if t_next in T_Nacht:
            # Wenn der Lkw NICHT auf Tour ist (also im Depot parkt)
            ist_im_depot = 1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

            for i in I:
                # Regel: Wenn er an Säule i steht, muss er dort stehen bleiben,
                # solange er im Depot ist (kein Umparken nachts).
                # Logik: Stecker(t, i) + ImDepot(t+1) - Stecker(t+1, i) <= 1
                # Das erzwingt: Wenn er zu t an i war und zu t+1 noch da ist,
                # MUSS er zu t+1 auch an i sein.
                scip.addCons(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + (1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t_next] for r in R))
                    - Zuordnung_LKW_Zeitpunkt_Charger[p, t_next, i] <= 1,
                    name=f"Nacht_Anschluss_Pflicht_{p}_{t}_{i}"
                )

print("6.")
print (scip.getNConss())
#Auswahl_Charger(c,i)
#1. Max 1 Charger Model pro Charger Platz
for i in I:
    # Jede Säule i (1, 2, 3) darf maximal einen Typ c haben
    scip.addCons(
        quicksum(Auswahl_Charger[c, i] for c in C) <= 1,
        name=f"Max_Ein_Typ_Pro_Saeule_{i}"
    )

print("7.")
print (scip.getNConss())
#Batteriespeicher
# 1. SOC-Berechnung
for t in T:
    t_prev = t - 1 if t > 0 else 95

    # Bilanz: Stand vorher + Laden - Entladen
    # Faktor 0.25 für 15-Min-Intervalle
    scip.addCons(
        SOC_Speicher[t] == SOC_Speicher[t_prev]
        + (Laden_Speicher[t] * 0.25 * 0.99)
        - (Entladen_Speicher[t] * 0.25 / 0.99),
        name=f"SOC_Bilanz_Speicher_t{t}"
    )

for t in T:
    # 1. Maximale Kapazität (nach oben gedeckelt durch Entscheidungsvariable)
    scip.addCons(SOC_Speicher[t] <= Batterie_kWh, name=f"Speicher_Max_Kappa_t{t}")

    # 2. Minimale Kapazität (DoD 97,5% -> 2,5% müssen drin bleiben)
    scip.addCons(SOC_Speicher[t] >= 0.025 * Batterie_kWh, name=f"Speicher_Min_DoD_t{t}")

for t in T:
    # Laden vom Netz begrenzt
    scip.addCons(Laden_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Laden_t{t}")

    # Entladen ins Depot begrenzt
    scip.addCons(Entladen_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Entladen_t{t}")

scip.addCons(SOC_Speicher[0] == SOC_Speicher[95], name="Speicher_Zyklus_Check")


# Peak Leistung
# 2. Constraints: P_peak muss größer sein als die Last in JEDEM Zeitintervall
# Wir gehen durch alle 96 Viertelstunden (t in T)
for t in T:
    # Berechne die tatsächliche Netzlast in diesem Intervall:
    # Last = (Summe aller LKW-Ladeleistungen) + (Speicher laden) - (Speicher entladen)
    aktuelle_netzlast = quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]

    # Wir fügen die Bedingung hinzu: P_peak >= aktuelle_netzlast
    scip.addCons(P_peak >= aktuelle_netzlast, name=f"Peak_Check_t{t}")

print("8.")
print (scip.getNConss())
# Rechenzeit optimierung
# Symmetrie-Brechung: Verhindert, dass der Solver identische Fahrzeug-IDs vertauscht
for i in range(len(P) - 1):
    p_current = P[i]
    p_next = P[i+1]
    # "Ein Modell für Fahrzeug i+1 darf nur gewählt werden, wenn auch für Fahrzeug i eines gewählt wurde"
    scip.addCons(
        quicksum(Flottenwahl[l, p_current] for l in L) >=
        quicksum(Flottenwahl[l, p_next] for l in L),
        name=f"Symmetry_Break_{p_current}_{p_next}"
    )

scip.getNConss()

1.
3600
2.
4020
3.
4041
4.
9821
5.
15965
6.
26693
7.
26696
8.
27273


27292

## Zielfunktion

In [46]:
# 1. Anteilige Fahrzeugkosten (pro Tag)
obj_veh = (1/260) * quicksum(
    Flottenwahl[l, p] * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L for p in P
)

# 2. Anteilige Kosten Ladeinfrastruktur (pro Tag)
obj_charger = (1/260) * (c_capex + quicksum(
    Auswahl_Charger[c, i] * (capex_ch[c] + opex_ch[c])
    for c in C for i in I
))

# 3. Anteilige Kosten Speicher (pro Tag)
obj_battery = (1/260) * (
    (Batterie_kWh * c_capex_bat_kWh) +
    (Batterie_kW * c_capex_bat_kW) +
    (Batterie_kWh * 0.02 * c_capex_bat_kWh) +
    (Batterie_kW * 0.02 * c_capex_bat_kW)
)

# 4. Anteilige Kosten Netzausbau (pro Tag)
obj_grid = (1/260) * (Ausbau_Netz * capex_grid_add)

# 5. Variable Kosten (Direkt pro Tag)

# Stromkosten (LKW + Speicher Laden - Speicher Entladen) * Umrechnung 15 min Intervall * Energiepreis
cost_electricity = quicksum(
    (quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]) *0.25 * c_energy
    for t in T
)

cost_peak = (1 / 260) * P_peak * c_peak

# Mautkosten NUR für Diesel (ActrosL)
# Wir koppeln das an die Variable Einsatz[r, l, p], da wir hier
# den LKW-Typ 'l' direkt prüfen können.
cost_toll = quicksum(
    Einsatz[r, 'ActrosL', p] * dist_toll[r] * c_toll
    for r in R for p in P if 'ActrosL' in L
)

# Dieselkosten (nur für Diesel-LKW 'ActrosL')
cost_diesel = quicksum(
    Einsatz[r, 'ActrosL', p] * (dist_total[r] * cons_d.get('ActrosL', 0) / 100.0) * diesel
    for r in R for p in P if 'ActrosL' in L
)

# GESAMT-ZIELFUNKTION
scip.setObjective(
    obj_veh + obj_charger + obj_battery + obj_grid + cost_electricity + c_peak + cost_toll + cost_diesel,
    "minimize"
)

## Berechnung des Ergebnisses

In [ ]:
#scip.setRealParam("limits/time", 30)
scip.redirectOutput()
scip.optimize()

presolving:
(round 1, fast)       401 del vars, 1584 del conss, 0 add conss, 3820 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 2, fast)       468 del vars, 1743 del conss, 0 add conss, 3941 chg bounds, 0 chg sides, 140 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 3, fast)       528 del vars, 1889 del conss, 0 add conss, 4001 chg bounds, 0 chg sides, 140 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 4, exhaustive) 528 del vars, 2934 del conss, 0 add conss, 4061 chg bounds, 20 chg sides, 340 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 5, exhaustive) 528 del vars, 2934 del conss, 0 add conss, 4101 chg bounds, 20 chg sides, 420 chg coeffs, 15068 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 6, medium)     528 del vars, 2955 del conss, 20 add conss, 4121 chg bounds, 40 chg sides, 500 chg coeffs, 15068 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 7, exhausti

In [ ]:
print ("Zielfunktions Wert Kosten: ",scip.getObjVal())

## Solution Output

In [ ]:
print("--- Fahrzeugmodell-Zuordnung ---")
for l in L:
    for p in P:
        if scip.getVal(Flottenwahl[l, p]) > 0.5:
            print(f"Fahrzeug ID {p}: Modell {l}")


print("\n--- Routen-Zuordnung (Route Assignment) ---")
for r in R:
    for p in P:
        if scip.getVal(Zuordnung_LKW_Route[r, p]) > 0.5:
            print(f"Route {r}: Gefahren von Fahrzeug ID {p}")

print("\n--- Auswahl Ladegeräte (Charger Selection) ---")
for c in C:
    for i in I:
        if scip.getVal(Auswahl_Charger[c, i]) > 0.5:
            print(f"Ladeplatz {i}: Modell {c}")

print("--- SOC für Fahrzeuge wo SOC >0 ---")
for p in P:
    for t in T:
        soc_val = scip.getVal(SOC_Zeit_Fahrzeug[p, t])
        if soc_val > 0:
          print(f"  Zeitintervall {t}; Fahrzeug: {p}; SOC={soc_val:.2f} kWh")

print("\n--- Ladeleistung----")
for p in P:
  for t in T:
    ladeleistung_val = scip.getVal(Ladeleistung[p, t])
    if ladeleistung_val > 0:
      print(f"Fahrzeug ID {p}, Zeitintervall {t}: Ladeleistung={ladeleistung_val:.2f} kW")

print("\n--- Netzausbau (Grid Expansion) ---")
if scip.getVal(Ausbau_Netz) > 0.5:
    print("Netzausbau: Ja")
else:
    print("Netzausbau: Nein")

print("\n--- Batteriespeicher (Battery Storage) ---")
if scip.getVal(Ausbau_Bat) > 0.5:
    print(f"Batteriespeicher gekauft: Ja")
    print(f"  Kapazität: {scip.getVal(Batterie_kWh):.2f} kWh")
    print(f"  Leistung: {scip.getVal(Batterie_kW):.2f} kW")
else:
    print("Batteriespeicher gekauft: Nein")


print("\n--- SOC und Laden/Entladen des Speichers (beispielhaft für die ersten 24 Zeitintervalle) ---")
for t in range(0, 24): # Display first 24 time intervals (6 hours)
    soc_storage = scip.getVal(SOC_Speicher[t])
    charge_storage = scip.getVal(Laden_Speicher[t])
    discharge_storage = scip.getVal(Entladen_Speicher[t])
    if soc_storage > 0:
      print(f"  Zeitintervall {t}: SOC_Speicher={soc_storage:.2f} kWh, Laden_Speicher={charge_storage:.2f} kW, Entladen_Speicher={discharge_storage:.2f} kW")

print("\n--- Batterie ---")
print(scip.getVal(Batterie_kWh))
print(scip.getVal(Batterie_kW))

In [ ]:
print("Model Fertig")